#### 处理连续值

In [1]:
density = [[0.697, True],[0.774, True],[0.634, True],[0.608, True],
           [0.556, True],[0.403, True],[0.481, True],[0.437, True],
          [0.666, False],[0.243, False],[0.245, False],[0.343, False],
          [0.639, False],[0.657, False],[0.360, False],[0.593, False],[0.719, False]]

In [2]:
from functools import cmp_to_key
# 根据密度排序
def sortByDensity(L) -> list:
    L.sort(key=lambda x:x[0])
    return L
# 计算每一个中间值 L已排序
def calcMidDensity(L) -> list:
    midDensity = []
    for i in range(len(density)-1):
        midDensity.append(float(format((density[i][0]+density[i+1][0])/2, '.3f')))
    return midDensity      

In [4]:
ListSorted = sortByDensity(density)
midDensity = calcMidDensity(ListSorted)
print(midDensity)
print(ListSorted)

[0.244, 0.294, 0.352, 0.382, 0.42, 0.459, 0.518, 0.575, 0.601, 0.621, 0.637, 0.648, 0.661, 0.681, 0.708, 0.746]
[[0.243, False], [0.245, False], [0.343, False], [0.36, False], [0.403, True], [0.437, True], [0.481, True], [0.556, True], [0.593, False], [0.608, True], [0.634, True], [0.639, False], [0.657, False], [0.666, False], [0.697, True], [0.719, False], [0.774, True]]


In [26]:
print('排序后的训练集')
for i in range(len(ListSorted)):
    if i%5 == 0 and i != 0:
        print(ListSorted[i])
    print(ListSorted[i], end='')
print('\n'+'-'*50)

print('所有的中位值：')
for i in range(len(midDensity)):
    if i%6 == 0 and i != 0:
        print(format(midDensity[i], '.3f'))
    print(str(format(midDensity[i], '.3f'))+'  ', end="")

排序后的训练集
[0.243, False][0.245, False][0.343, False][0.36, False][0.403, True][0.437, True]
[0.437, True][0.481, True][0.556, True][0.593, False][0.608, True][0.634, True]
[0.634, True][0.639, False][0.657, False][0.666, False][0.697, True][0.719, False]
[0.719, False][0.774, True]
--------------------------------------------------
所有的中位值：
0.244  0.294  0.352  0.382  0.420  0.459  0.518
0.518  0.575  0.601  0.621  0.637  0.648  0.661
0.661  0.681  0.708  0.746  

In [17]:
import math
#求总样本的信息增益
def calcEnt(List) -> float:
    positive = 0
    negative = 0
    number = len(List)
    for item in List:
        if item[1] == True:
            positive += 1
        else:
            negative += 1
    pp = positive / number
    pn = negative / number
    if pp == 0:
        pp = 1
    if pn == 0:
        pn = 1
    ent = pp * math.log(pp, 2) + pn * math.log(pn, 2)
    return format(-ent, '.3f')

In [18]:
print(calcEnt(density))
print(type(midDensity[-0]))

0.998
<class 'float'>


In [19]:
# 计算条件熵
def calcDt(List, number, flag) -> float:
    positive = 0
    negative = 0
    if flag == 0:
        #计算小于划分值部分
        for i in range(number):
            if List[i][1] == True:
                positive += 1
            else:
                negative += 1
        pp = positive / number
        pn = negative / number
        if pp == 0:
            pp = 1
        if pn == 0:
            pn = 1
        dt0 = pp * math.log(pp, 2) + pn * math.log(pn, 2)
        return format(-dt0, '.3f')
    #计算大于划分值部分
    for i in range(len(List)-number, len(List)):
            if List[i][1] == True:
                positive += 1
            else:
                negative += 1
    pp = positive / number
    pn = negative / number
    if pp == 0:
        pp = 1
    if pn == 0:
        pn = 1
    dt1 = pp * math.log(pp, 2) + pn * math.log(pn, 2)
    return format(-dt1, '.3f')
            
            
# 计算不同划分下信息增益 List 已排序
def calcGain(List, midDensity):
    
    ent = calcEnt(List)
    number = len(List)
    gainList = [] #保存每一种划分情况下的信息增益
    maxGain = -1  #标记最大的信息增益
    maxMidData = 0 #标记使信息增益最大的划分值
    for midData in midDensity:
        number0 = 0
        number1 = 0
        for j in range(len(List)):
            if List[j][0] < midData:
                number0 += 1
            else:
                number1 += 1
        # 计算每一种划分值下的条件熵以及信息增益
        dt0 = calcDt(List, number0, 0)
        dt1 = calcDt(List, number1, 1)
        dt = float(number0 / number) * float(dt0) + float(number1 / number) * float(dt1)
        gain = float(ent) - float(dt)
        gainList.append(float(format(gain, '.4f')))
        if gain > maxGain:
            maxGain = gain
            maxMidData = midData
    return gainList, maxMidData

In [25]:
gainList, maxMidData = calcGain(ListSorted, midDensity)
print('每个划分值对应的信息增益为：')
for i in range(len(gainList)):
    if i%6 == 0 and i != 0:
        print(format(gainList[i], '.3f'))
    print(str(format(gainList[i], '.3f'))+'  ', end="")

print('\n信息增益最大的划分值为:'+str(maxMidData))

每个划分值对应的信息增益为：
0.057  0.118  0.187  0.263  0.094  0.031  0.004
0.004  0.003  0.003  0.004  0.031  0.006  0.001
0.001  0.025  0.001  0.067  
信息增益最大的划分值为:0.382
